In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import category_encoders as ce
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, mean_absolute_error
import scipy.stats as stats

print(np.__version__)

1.24.3


In [7]:
pip install catboost --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.1/27.1 MB 4.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lightgbm --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
#: Import CatBoost or LightGBM and run your models
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor





[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [25]:
# Load your data
#cleaned dataset
df = pd.read_csv("auto_data_cleaned_16_09_2024.csv")
# Example setup of columns, adjust based on your dataset structure

df=df.drop(columns=['year', 'offer_description'])
df = df.dropna(subset=['color'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238221 entries, 0 to 238301
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   brand                     238221 non-null  object 
 1   model                     238221 non-null  object 
 2   color                     238221 non-null  object 
 3   price_in_euro             238221 non-null  float64
 4   power_ps                  238221 non-null  float64
 5   transmission_type         238221 non-null  object 
 6   fuel_type                 238221 non-null  object 
 7   fuel_consumption_l_100km  238221 non-null  float64
 8   mileage_in_km             238221 non-null  float64
 9   age                       238221 non-null  float64
dtypes: float64(5), object(5)
memory usage: 20.0+ MB


## Catboost model training:

In [27]:
# Load your data
#cleaned dataset
# setup of features
target = 'price_in_euro'
numerical_features = ['age', 'power_ps', 'fuel_consumption_l_100km', 'mileage_in_km']
categorical_features = ['color', 'model', 'brand', 'fuel_type', 'transmission_type']

# Splitting data into X and y
X = df[numerical_features + categorical_features]
y = df[target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### CatBoost Model ###
print("Training CatBoost Model...")
catboost_model = CatBoostRegressor(cat_features=categorical_features, verbose=0)  # Setting categorical features
catboost_model.fit(X_train, y_train)

# Make predictions and evaluate CatBoost
y_pred_catboost = catboost_model.predict(X_test)
mae_catboost = mean_absolute_error(y_test, y_pred_catboost)
print(f"CatBoost MAE: {mae_catboost}")

test_r2 = r2_score(y_test, y_pred_catboost)
print(f"Test R² - full dataset: {test_r2}")
mse_test = mean_squared_error(y_test, y_pred_catboost)
print(f"Test Set MSE - full dataset: {mse_test}")


Training CatBoost Model...
CatBoost MAE: 3805.5119981235016
Test R² - full dataset: 0.8779305478439742
Test Set MSE - full dataset: 142725576.05539766


### Dataset without extreme price values

In [61]:
# Load your data 
#cleaned dataset
df = pd.read_csv("auto_data_cleaned_16_09_2024.csv")

#  setup of features, adjust based on your dataset structure

df=df.drop(columns=['year', 'offer_description'])
df = df.dropna(subset=['color'])


# Remove outliers using Z-score for price and mileage
df_clean = df[(np.abs(stats.zscore(df['price_in_euro'])) < 9)]  # 4 is the higher threshold for outliers
df_clean = df_clean[(np.abs(stats.zscore(df_clean['mileage_in_km'])) < 4)]  # Do the same for mileage

In [65]:
# Example setup of columns, adjust based on your dataset structure
target = 'price_in_euro'
numerical_features = ['age', 'power_ps', 'fuel_consumption_l_100km', 'mileage_in_km']
categorical_features = ['color', 'model', 'brand', 'fuel_type', 'transmission_type']

# Splitting data into X and y
X = df_clean[numerical_features + categorical_features]
y = df_clean[target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### CatBoost Model ###
print("Training CatBoost Model...")
catboost_model = CatBoostRegressor(cat_features=categorical_features, verbose=0, n_estimators=500, max_depth=None, grow_policy='Depthwise', bagging_temperature=100)  # Setting categorical features
catboost_model.fit(X_train, y_train)

# Make predictions and evaluate CatBoost
y_pred_catboost = catboost_model.predict(X_test)
mae_catboost = mean_absolute_error(y_test, y_pred_catboost)
print(f"CatBoost MAE: {mae_catboost}")

test_r2 = r2_score(y_test, y_pred_catboost)
print(f"Test R² - full datase withput Outliers: {test_r2}")
mse_test = mean_squared_error(y_test, y_pred_catboost)
print(f"Test Set MSE - full dataset without Outliers: {mse_test}")


Training CatBoost Model...
CatBoost MAE: 3263.3470176493884
Test R² - full datase withput Outliers: 0.929184192123229
Test Set MSE - full dataset without Outliers: 48727422.47352347


### Without colors:

In [66]:
# Load your data 
#cleaned dataset
df = pd.read_csv("auto_data_cleaned_16_09_2024.csv")

#  setup of features, adjust based on your dataset structure

df=df.drop(columns=['color', 'year', 'offer_description'])


# Remove outliers using Z-score for price and mileage
df_clean = df[(np.abs(stats.zscore(df['price_in_euro'])) < 9)]  # 4 is the higher threshold for outliers
df_clean = df_clean[(np.abs(stats.zscore(df_clean['mileage_in_km'])) < 4)]  # Do the same for mileage

In [67]:
# Example setup of columns, adjust based on your dataset structure
target = 'price_in_euro'
numerical_features = ['age', 'power_ps', 'fuel_consumption_l_100km', 'mileage_in_km']
categorical_features = [ 'model', 'brand', 'fuel_type', 'transmission_type']

# Splitting data into X and y
X = df_clean[numerical_features + categorical_features]
y = df_clean[target]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### CatBoost Model ###
print("Training CatBoost Model...")
catboost_model = CatBoostRegressor(cat_features=categorical_features, verbose=0, n_estimators=500, max_depth=None, grow_policy='Depthwise', bagging_temperature=100)  # Setting categorical features
catboost_model.fit(X_train, y_train)

# Make predictions and evaluate CatBoost
y_pred_catboost = catboost_model.predict(X_test)
mae_catboost = mean_absolute_error(y_test, y_pred_catboost)
print(f"CatBoost MAE: {mae_catboost}")

test_r2 = r2_score(y_test, y_pred_catboost)
print(f"Test R² - full datase withput Outliers: {test_r2}")
mse_test = mean_squared_error(y_test, y_pred_catboost)
print(f"Test Set MSE - full dataset without Outliers: {mse_test}")


Training CatBoost Model...
CatBoost MAE: 3288.682265481608
Test R² - full datase withput Outliers: 0.9182634710590439
Test Set MSE - full dataset without Outliers: 54985339.42732238


## Lightgbm model:

In [32]:
#cleaned dataset
df = pd.read_csv("auto_data_cleaned_16_09_2024.csv")

# the models will be encoded by mean value of price of every model
target_encoder = ce.TargetEncoder(cols=['model'])
df['Modell_encoded'] = target_encoder.fit_transform(df['model'], df['price_in_euro'])  # 'Preis' ist die Zielvariable


# the brand will be encoded by mean value of price of every model
target_encoder = ce.TargetEncoder(cols=['brand'])
df['Brand_encoded'] = target_encoder.fit_transform(df['brand'], df['price_in_euro'])  # 'Preis' ist die Zielvariable

df=df.drop(columns=['year', 'offer_description', ])

df = pd.get_dummies(df, columns=[ 'fuel_type', 'transmission_type','color'], dtype='int')


# Splitting the data into training and test sets
X = df.drop(['price_in_euro', 'model', 'brand'], axis=1)  # Features
y = df['price_in_euro']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
### LightGBM Model ###
# Encoding categorical variables for LightGBM since it doesn’t handle string categories directly


print("Training LightGBM Model...")
lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)

# Make predictions and evaluate LightGBM
y_pred_lgbm = lgbm_model.predict(X_test)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
print(f"LightGBM MAE: {mae_lgbm}")


test_r2_lgbm = r2_score(y_test, y_pred_lgbm)
print(f"Test R² - full datase : {test_r2_lgbm}")
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
print(f"Test Set MSE - full dataset : {mse_lgbm}")


#

Training LightGBM Model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001973 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1054
[LightGBM] [Info] Number of data points in the train set: 190641, number of used features: 31
[LightGBM] [Info] Start training from score 25715.360715
LightGBM MAE: 3928.7472695796605
Test R² - full datase : 0.8709635363752204
Test Set MSE - full dataset : 146176671.17697877


### Dataset without extreme price values

In [36]:
#cleaned dataset
df = pd.read_csv("auto_data_cleaned_16_09_2024.csv")

# the models will be encoded by mean value of price of every model
target_encoder = ce.TargetEncoder(cols=['model'])
df['Modell_encoded'] = target_encoder.fit_transform(df['model'], df['price_in_euro'])  # 'Preis' ist die Zielvariable


# the brand will be encoded by mean value of price of every model
target_encoder = ce.TargetEncoder(cols=['brand'])
df['Brand_encoded'] = target_encoder.fit_transform(df['brand'], df['price_in_euro'])  # 'Preis' ist die Zielvariable

df=df.drop(columns=['year', 'offer_description', ])

df = pd.get_dummies(df, columns=[ 'fuel_type', 'transmission_type','color'], dtype='int')


# Remove outliers using Z-score for price and mileage
df_clean = df[(np.abs(stats.zscore(df['price_in_euro'])) < 5)]  # 4 is the higher threshold for outliers
df_clean = df_clean[(np.abs(stats.zscore(df_clean['mileage_in_km'])) < 4)]  # Do the same for mileage


# Splitting the data into training and test sets
X = df_clean.drop(['price_in_euro', 'model', 'brand'], axis=1)  # Features
y = df_clean['price_in_euro']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
### LightGBM Model ###
# Encoding categorical variables for LightGBM since it doesn’t handle string categories directly


print("Training LightGBM Model...")
lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)

# Make predictions and evaluate LightGBM
y_pred_lgbm = lgbm_model.predict(X_test)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
print(f"LightGBM MAE: {mae_lgbm}")


test_r2_lgbm = r2_score(y_test, y_pred_lgbm)
print(f"Test R² - full dataset without Outliers : {test_r2_lgbm}")
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
print(f"Test Set MSE - full dataset without Outliers : {mse_lgbm}")


#

Training LightGBM Model...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1057
[LightGBM] [Info] Number of data points in the train set: 189092, number of used features: 31
[LightGBM] [Info] Start training from score 23860.429669
LightGBM MAE: 3286.47428153585
Test R² - full dataset without Outliers : 0.9003591242450679
Test Set MSE - full dataset without Outliers : 42454646.65125012


In [ ]:
!pip install numpy --upgrade --quiet

# Step 4: Upgrade my numpy back to the latest version (if needed)
#!pip install numpy --upgrade --quiet

# Step 5: Restart the kernel again to apply the numpy upgrade
